In [ ]:
#

# Show and move preprocessed dirs

The script is used to select best Ven and Art CT image of each animal. The input is directory with CT images converted to `nii.gz`.

In [ ]:
# reload modules
%load_ext autoreload
%autoreload 2

from pathlib import Path
import os


In [ ]:
# read from .env file or environment variables
# from dotenv import load_dotenv
PILSEN_PIGS_CT_DIR = Path("~/bmc_ct").expanduser()
if not PILSEN_PIGS_CT_DIR.exists():
    PILSEN_PIGS_CT_DIR = Path("~/mnt/nas-bmc3_ct").expanduser()

preprocessed_dirs = [PILSEN_PIGS_CT_DIR / r"pilsen_pigs_all/transplantation_nii_transposed/preprocessed/Prasata_C.tmp_need_more_descriptive_names"]
preprocessed_dirs = [PILSEN_PIGS_CT_DIR / r"pilsen_pigs_all/transplantation_nii_transposed/preprocessed/Prasata_DC"]
preprocessed_dir = preprocessed_dirs[0]

DEST_ROOT = PILSEN_PIGS_CT_DIR / "pilsen_pigs_all/transplantation_nii_transposed/Prasata_DC"


SOURCE_ROOT = preprocessed_dir
assert SOURCE_ROOT.exists(), f"Zdrojový adresář neexistuje: {SOURCE_ROOT}"

In [ ]:



import os
import re
import glob
import shutil
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from collections import defaultdict

# SOURCE_ROOT = "/cesta/k/adresarove/strukture"
# DEST_ROOT = "/cesta/kam/kopirovat"

# regex pro kód, např. C11, TX12, AB7
CODE_REGEX = re.compile(r"[A-Z]+[0-9]+")

# najdi všechny adresáře v rootu
all_dirs = [d for d in glob.glob(os.path.join(SOURCE_ROOT, "*")) if os.path.isdir(d)]

# seskup podle kódu
groups = defaultdict(list)
for d in all_dirs:
    m = CODE_REGEX.search(os.path.basename(d))
    if m:
        groups[m.group()].append(d)

# vypiš nalezené kódy
len_groups = len(groups)
print(f"Nalezeno {len_groups} kódů:")

for i, code in enumerate(groups.keys()):
    print(f"{i}: {code} ({len(groups[code])} adresářů)")

if len_groups == 0:
    print(all_dirs)
# vyber kód
# choice = int(input("Zadej číslo kódu: "))
# print(f"{choice=}")
# selected_code = list(groups.keys())[choice]
# print(f"{selected_code=}")
# dirs = groups[selected_code]
# print(f"{len(dirs)=}")
#
#
# # načti obrázky
# images = []
# for d in dirs:
#     d = Path(d)
#     # print(f"{d=}")
#     assert d.exists(), f"Adresář neexistuje: {d}"
#     files = list(d.glob("*sagittal.jpg"))
#     if files:
#         images.append((d, files[0]))
#
# # zobraz grid
# cols = 4
# rows = (len(images) + cols - 1) // cols
# print(f"{rows=}")
# fig, axes = plt.subplots(rows, cols, figsize=(18, 6*rows))
# axes = axes.flatten()
#
# for i, (d, img_path) in enumerate(images):
#     img = mpimg.imread(img_path)
#     axes[i].imshow(img)
#     axes[i].axis("off")
#     axes[i].set_title(f"{i}: {os.path.basename(d)}", fontsize=8)
#
# for j in range(i+1, len(axes)):
#     axes[j].axis("off")
#
# plt.tight_layout()
# plt.show()


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

codes = list(groups.keys())

dropdown = widgets.Dropdown(
    options=codes,
    description='Kód:',
    value=codes[0]
)

out = widgets.Output()

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        with out:
            clear_output(wait=True)
            code = change['new']
            dropdown.selected_code = change['new']
            dirs = groups[code]

            images = []
            for d in dirs:
                files = list(Path(d).glob("*sagittal.jpg"))
                if files:
                    images.append((d, files[0]))

            dropdown.images = images
            print(f"{len(dropdown.images)=}")

            cols = 4
            rows = (len(images) + cols - 1) // cols
            fig, axes = plt.subplots(rows, cols, figsize=(18, 6*rows))
            axes = axes.flatten()

            for i, (d, img_path) in enumerate(images):
                img = mpimg.imread(img_path)
                axes[i].imshow(img)
                axes[i].axis("off")
                axes[i].set_title(f"{i}: {os.path.basename(d)}", fontsize=8)

            for j in range(i+1, len(axes)):
                axes[j].axis("off")

            plt.tight_layout()
            plt.show()

dropdown.observe(on_change)

display(dropdown, out)

In [ ]:
selected_code = dropdown.selected_code
print(f"{dropdown.selected_code=}")
# print(f"{dropdown.images=}")
selected_code


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import shutil
from pathlib import Path

# DEST_ROOT = Path("/cesta/kam/presunout")
DEST_ROOT.mkdir(parents=True, exist_ok=True)

selected_code = dropdown.selected_code
images = dropdown.images


# vybrané indexy (např. z předchozího výběru)
selection = input("Zadej čísla adresářů k přesunutí (oddělené čárkou): ")
indices = [int(x.strip()) for x in selection.split(",") if x.strip().isdigit()]

# připrav seznam textových polí pro nové názvy
text_inputs = []
for ind, i in enumerate(indices):
    src_dir = Path(images[i][0])
    art_ven = "Art" if ind == 0 else "Ven" if ind == 1 else ""
    default_name = f"{selected_code}_{i:03d}"
    default_name = f"{selected_code}_{art_ven}"
    txt = widgets.Text(
        value=default_name,
        description=os.path.basename(src_dir),
        layout=widgets.Layout(width="500px")
    )
    text_inputs.append((src_dir, txt))

# tlačítko pro spuštění přesunu
button = widgets.Button(description="Přesunout vybrané")

output = widgets.Output()

def on_click(b):
    with output:
        clear_output()
        for src_dir, txt in text_inputs:
            orig_name = src_dir.name
            new_name = txt.value.strip()
            dest_dir = DEST_ROOT / new_name

            print(f"Přesouvám {src_dir} -> {dest_dir}")
            shutil.move(str(src_dir), dest_dir)

            note_path = dest_dir / "note.txt"
            with open(note_path, "w", encoding="utf-8") as f:
                f.write(f"Original name: {orig_name}\n")

            list(dest_dir.glob("*.nii.gz"))[0].rename(dest_dir / f"{new_name}.nii.gz")

# propojení
button.on_click(on_click)

# zobraz widgety
for _, txt in text_inputs:
    display(txt)
display(button, output)